# Otimização de Consultas no Spark

Adaptive Query Execution (AQE) é um dos melhores recursos do Spark 3.0, que reotimiza e ajusta os planos de consulta com base nas estatísticas de tempo de execução coletadas durante a execução da consulta. 

Depois de habilitar a AQE, as seguintes melhorias serão realizadas:

 - Conversão automatica do sort-merge join (lento) para o Broadcast join
 - Otimização do Skew Join (dados distribuidos de forma desigual entre as partições no cluster)


para habilitaro AQS: 

* spark.conf.set("spark.sql.adaptive.enabled", "true")



In [1]:
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F

spark = SparkSession\
    .builder\
    .appName("Spark Performance Issues")\
    .config("spark.sql.adaptive.enabled", "false")\
    .config("spark.metrics.conf.*.sink.console.class", "org.apache.spark.metrics.sink.ConsoleSink")\
    .getOrCreate()

sql_context = SQLContext(sc)

### Simulando um desbalanceamento entre as partições (Data Skew)


Data Skew é uma condição em que os dados de uma tabela são distribuídos de forma desigual entre as partições no cluster. A distorção de dados pode prejudicar gravemente o desempenho das consultas, especialmente aquelas com junções. As junções entre tabelas grandes exigem dados embaralhados (shuffling) e a distorção pode levar a um  desequilíbrio extremo de trabalho no cluster. É provável que a distorção de dados esteja afetando a performance de uma consulta caso ela pareça estar travada ao termino de poucas tasks (por exemplo, as últimas 3 tasks de 200). 


In [2]:
spark.conf.set("spark.sql.shuffle.partitions", 10)

spark.range(1000000)\
    .withColumn("join_key", F.lit(" "))\
    .createOrReplaceTempView("table_x")

spark.range(1000000)\
    .withColumn("join_key", F.lit(" "))\
    .createOrReplaceTempView("table_y")


No código a seguir, será realizado o join entre as duas tabelas, o que irá produzir um output de um trilhão de linhas, e todas elas serão produzidas utilizando um único executor (o executor que obtém a chave ""):

Esta consulta parece estar em execução. Mas, com apenas uma única task remanescente, quando dispararmos a action. O job permanecerá em execução por um longo período, até falhar. Nesse caso, há apenas uma chave de join problemática. Em outros cenários, podem existir outros fatores.

In [ ]:
query = """
    SELECT id, count()
    FROM
    (
        SELECT x.id
        FROM table_x x
        JOIN table_y y ON x.join_key = y.join_key
    )
    GROUP BY id
"""

df = sql_context.sql(query)

In [ ]:
df.show()

In [ ]:
# analisa a distribuição dos dados nas das partições
for i, part in enumerate(df.rdd.glom().collect()):
    print({i: part})


In [ ]:
'''

Para resolver o problema de distorção de dados, podemos:

* Reparticionar os dados utilizando uma chave distribuída de maneira mais uniforme.
* Transmitir (Broadcast) o dataframe menor
* Dividir os dados em dados distorcidos e não distorcidos e trabalhar com eles em paralelo redistribuindo os dados distorcidos (replicação diferencial)
* Use uma chave aleatória adicional para melhorar a distribuição dos dados (salting).
* Usar o broadcast join

PS: no plano de execução, observe os operadores Exchange e o scan (como podemos melhorar isso?)

'''


In [ ]:
df.explain()